<div align="center">

This is notebook  for ML Project 

</div>

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [16]:
# !git clone https://github.com/ultralytics/yolov5  # clone
#%cd yolov5
! pip install -r requirements.txt  
# install

import torch
import utils
display = utils.notebook_init()  # checks

AttributeError: module 'utils' has no attribute 'notebook_init'

In [17]:
import os

ImagePath = "./dataset/images/"        # images (relative to 'path')
LabelPath = "./dataset/labels/"        # labels (relative to 'path')

trainImages = os.path.join(ImagePath, 'train')
validImages = os.path.join(ImagePath, 'val')
trainlabels = os.path.join(LabelPath, 'train')
validlabels = os.path.join(LabelPath, 'val')

train_data=[trainImages,trainlabels]
val_data=[validImages,validlabels]

print("# of train Images: ", len(os.listdir(trainImages)))
print("# of valid Images: ", len(os.listdir(validImages)))
print("# of train Labels: ", len(os.listdir(trainlabels)))
print("# of valid Labels: ", len(os.listdir(validlabels)))

FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset/images/train'

In [ ]:
%echo %cd%

# Train

Train a YOLOv5s Classification model on the dataset with

In [3]:
!python train.py --img 640 --batch 16 --epochs 5 --data cocoCustomData.yaml --weights yolov5s.pt --nosave --cache


train: weights=yolov5s.pt, cfg=, data=cocoCustomData.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'F:\ML': No such file or directory
YOLOv5  2024-3-25 Python-3.11.9 torch-2.2.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05

# Test

test on a video

In [19]:
#!python detect.py --weights runs/train/exp6/weights/best.pt --conf 0.25 --source ../vid.mp4 --save-crop

^C


detect: weights=['runs/train/exp6/weights/best.pt'], source=../vid.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'F:\ML': No such file or directory
YOLOv5  2024-3-25 Python-3.12.2 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/6826) F:\ML project\vid.mp4: 384x640 (no detections), 333.3ms
video 1/1 (2/6826) F:\ML project\vid.mp4: 384x640 (no detections), 294.3ms
video 1/1 (3/6826) F:\ML project\vid.mp4: 384x640 (no detections), 312.1ms
video 1/1 (4/6826) F:\ML project\vid.mp4: 384x640 (no detections), 299.6ms
video 1/1 (5/682

In [20]:
import os

ImagePath = "../dataset/images/"        # images (relative to 'path')
LabelPath = "../dataset/labels/"        # labels (relative to 'path')

trainImages = os.path.join(ImagePath, 'train')
validImages = os.path.join(ImagePath, 'val')
trainlabels = os.path.join(LabelPath, 'train')
validlabels = os.path.join(LabelPath, 'val')

#train_data=[trainImages,trainlabels]
#val_data=[validImages,validlabels]

tlst = os.listdir(trainImages)
vlst = os.listdir(trainlabels)

print(os.listdir(trainImages))
print(os.listdir(validImages))

# for im in tlst:
#     !python detect.py --weights runs/train/exp6/weights/best.pt --img 640 --conf 0.25 --source im --save-crop

# for im in vlst:
#     !python detect.py --weights runs/train/exp6/weights/best.pt --img 640 --conf 0.25 --source im --save-crop
    
# print("# of train Images: ", len(os.listdir(trainImages)))
# print("# of valid Images: ", len(os.listdir(validImages)))
# print("# of train Labels: ", len(os.listdir(trainlabels)))
# print("# of valid Labels: ", len(os.listdir(validlabels)))

['BloodImage_00000.jpg', 'BloodImage_00001.jpg', 'BloodImage_00002.jpg', 'BloodImage_00003.jpg', 'BloodImage_00004.jpg', 'BloodImage_00005.jpg', 'BloodImage_00006.jpg', 'BloodImage_00007.jpg', 'BloodImage_00008.jpg', 'BloodImage_00009.jpg', 'BloodImage_00010.jpg', 'BloodImage_00011.jpg', 'BloodImage_00012.jpg', 'BloodImage_00013.jpg', 'BloodImage_00014.jpg', 'BloodImage_00015.jpg', 'BloodImage_00016.jpg', 'BloodImage_00017.jpg', 'BloodImage_00018.jpg', 'BloodImage_00019.jpg', 'BloodImage_00020.jpg', 'BloodImage_00021.jpg', 'BloodImage_00022.jpg', 'BloodImage_00023.jpg', 'BloodImage_00024.jpg', 'BloodImage_00026.jpg', 'BloodImage_00028.jpg', 'BloodImage_00029.jpg', 'BloodImage_00030.jpg', 'BloodImage_00031.jpg', 'BloodImage_00032.jpg', 'BloodImage_00033.jpg', 'BloodImage_00034.jpg', 'BloodImage_00035.jpg', 'BloodImage_00036.jpg', 'BloodImage_00037.jpg', 'BloodImage_00038.jpg', 'BloodImage_00039.jpg', 'BloodImage_00040.jpg', 'BloodImage_00041.jpg', 'BloodImage_00042.jpg', 'BloodImage_000

In [3]:
#!python detect.py --weights runs/train/exp6/weights/best.pt --img 640 --conf 0.25 --source ../BI.jpg

detect: weights=['runs/train/exp6/weights/best.pt'], source=../BI.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'F:\ML': No such file or directory
YOLOv5  2024-3-25 Python-3.12.2 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 F:\ML project\BI.jpg: 480x640 25 RBCs, 196.0ms
Speed: 2.0ms pre-process, 196.0ms inference, 12.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp7


then split the images and return the images detected to be further processed.

In [5]:
%cd yolov5
!python detect.py --weights runs/train/exp6/weights/best.pt --img 640 --conf 0.25 --source ../BI.jpg --save-crop

f:\ML project\yolov5


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
detect: weights=['runs/train/exp6/weights/best.pt'], source=../BI.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'F:\ML': No such file or directory
YOLOv5  2024-3-25 Python-3.12.2 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 F:\ML project\BI.jpg: 480x640 25 

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import copy
import matplotlib.pyplot as plt

def create_lenet():
    model = nn.Sequential(
        nn.Conv2d(3, 6, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),

        nn.Conv2d(6, 16, kernel_size=5, padding=0),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),

        nn.Flatten(),
        nn.Linear(16 * 88 * 88, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )
    return model

In [27]:
def validate(model, data):
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in data:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct * 100.0 / total



In [28]:
def evaluate(model, val_loader, device):
  model.eval()
  running_loss = 0.0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      running_loss += loss.item()

  val_loss = running_loss / len(val_loader)
  model.train()

  return val_loss


In [30]:
def train(train_loader, val_loader, numb_epochs=10, lr=0.001, device="cpu"):
  # Create the LeNet model
  model = create_lenet()
  model.to(device)
  best_model = None
  min_val_loss = 0
  accuracies = []

  # Define loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Adjust learning rate as needed

  # Training loop
  for epoch in range(numb_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
      images, labels = images.to(device), labels.to(device)

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Update running loss
      running_loss += loss.item()

    # Calculate average training loss
    epoch_loss = running_loss / len(train_loader)

    # Perform validation (optional)
    val_loss = evaluate(model, val_loader, device)
    accuracy = validate(cnn, val_dl)
    accuracies.append(accuracy)
    if accuracy > max_accuracy:
      best_model = copy.deepcopy(cnn)
      max_accuracy = accuracy
      print("Saving best model with accuracy: ", accuracy)
    
    # Print training and validation loss
    print(f"Epoch {epoch+1}/{numb_epochs} - Training Loss: {epoch_loss:.3f} - Val Loss: {val_loss:.3f} - Accuracy: {accuracy:.3f}")
  
  plt.plot(accuracies)
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.title('Validation Accuracy')
  plt.show()

  return best_model


In [31]:
def predict(model, data_loader, device="cpu"):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions


In [32]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [34]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, images_path, image_size=(224, 224)):
        self.images_path = images_path
        #self.labels_path = labels_path
        self.image_size = image_size
        self.classes = sorted(os.listdir(images_path))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self._load_images()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path, class_name = self.images[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = cv2.resize(image, self.image_size)  # Resize image
        image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]
        image = np.transpose(image, (2, 0, 1))  # Transpose to (C, H, W)
        label = self.class_to_idx[class_name]
        return torch.tensor(image), torch.tensor(label)

    def _load_images(self):
        images = []
        for class_name in self.classes:
            class_path = os.path.join(self.images_path, class_name)
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                images.append((image_path, class_name))
        return images

def CustomDataLoader(data, batch_size=32, image_size=(224, 224)):
    images_path = data
    dataset = CustomDataset(images_path, image_size=image_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Usage example:
# wbc_data_path = "./lenetData/wbc/"
# wbc_dl = CustomDataLoader(wbc_data_path, batch_size=numb_batch)


In [35]:
import os

ImagePath = "./lenetData/wbc/"        # images (relative to 'path')

wbcTypes=["basophil","eosinophil","erythroblast","ig","lymphocyte","monocyte","neutrophil","platelet"]

for i in wbcTypes:
    _path = os.path.join(ImagePath, i)
    print("# of train Images of ",i,": ", len(os.listdir(_path)))


# of train Images of  basophil :  1218
# of train Images of  eosinophil :  3117
# of train Images of  erythroblast :  1551
# of train Images of  ig :  2895
# of train Images of  lymphocyte :  1214
# of train Images of  monocyte :  1420
# of train Images of  neutrophil :  3330
# of train Images of  platelet :  2348


In [36]:
numb_batch=10
device="cpu"

In [37]:
# importing the module 
import PIL 
from PIL import Image 
  
# loading the image 
img = PIL.Image.open("Blue.jpg") 
  
# fetching the dimensions 
wid, hgt = img.size 
  
# displaying the dimensions 
print(str(wid) + "x" + str(hgt))

360x363


wbc lenet

In [45]:
# Paths
ImagePath = "./lenetData/wbc/"
wbcTypes = sorted(os.listdir(ImagePath))

# Data loader
train_dl = CustomDataLoader(ImagePath, batch_size=numb_batch, image_size=(360, 363))
val_dl = CustomDataLoader(ImagePath, batch_size=numb_batch, image_size=(360, 363))

# Training
wbc_lenet = train(train_dl, val_dl, numb_epochs=1, device=device)


rbc lenet

In [ ]:
# Paths
ImagePath = "./lenetData/rbc/"
wbcTypes = sorted(os.listdir(ImagePath))

# Data loader
train_dl = CustomDataLoader(ImagePath, batch_size=numb_batch)
val_dl = CustomDataLoader(ImagePath, batch_size=numb_batch)

# Training
rbc_lenet = train(train_dl, val_dl, numb_epochs=10, device=device)


pletelet lenet

In [ ]:
# Paths
ImagePath = "./lenetData/plateletes/"
wbcTypes = sorted(os.listdir(ImagePath))

# Data loader
train_dl = CustomDataLoader(ImagePath, batch_size=numb_batch)
val_dl = CustomDataLoader(ImagePath, batch_size=numb_batch)

# Training
plateletes_lenet = train(train_dl, val_dl, numb_epochs=10, device=device)


In [ ]:
import cv2
import torch
from yolov5.detect import detect_objects # Assuming you have a function to detect objects using YOLOv5
from lenet import lenet_wbc, lenet_rbc, lenet_platelets # Assuming you have separate Lenet models for each cell type

# Load YOLOv5 model
yolov5_model = torch.load('path_to_yolov5_model.pth')

# Load Lenet models
lenet_wbc_model = torch.load('path_to_wbc_lenet_model.pth')
lenet_rbc_model = torch.load('path_to_rbc_lenet_model.pth')
lenet_platelets_model = torch.load('path_to_platelets_lenet_model.pth')

# Load image
image = cv2.imread('path_to_image.jpg')

# Detect objects using YOLOv5
detected_objects = detect_objects(yolov5_model, image)

# Dictionary to store cropped images and their confidence values
cropped_images = {'wbc': [], 'rbc': [], 'platelets': []}

# Crop out images and pass them to respective Lenet models
for obj in detected_objects:
    class_name, confidence, bbox = obj['class'], obj['confidence'], obj['bbox']
    cropped_image = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]  # Cropping out the detected object
    
    # Pass cropped image to respective Lenet model based on class name
    if class_name == 'wbc':
        output = lenet_wbc(cropped_image)  # Assuming lenet_wbc is a function that predicts size of WBC
        cropped_images['wbc'].append((cropped_image, confidence, output))
    elif class_name == 'rbc':
        output = lenet_rbc(cropped_image)  # Assuming lenet_rbc is a function that predicts size of RBC
        cropped_images['rbc'].append((cropped_image, confidence, output))
    elif class_name == 'platelets':
        output = lenet_platelets(cropped_image)  # Assuming lenet_platelets is a function that predicts size of platelets
        cropped_images['platelets'].append((cropped_image, confidence, output))

# Print information
for cell_type, data in cropped_images.items():
    print(f"Detected {cell_type}:")
    for cropped_image, confidence, size_prediction in data:
        print(f"Confidence: {confidence}, Size Prediction: {size_prediction}")

